In [22]:
# importing sqlite to connect to the database
# importing pandas for data manipulation
# importing requests to get data from the api
import pandas as pd
import sqlite3
import numpy as np
import requests

In [23]:
# Feature Engineering for FPL predictiions
# Creating predictive feature from player data

# Connecting to the database
conn = sqlite3.connect('fpl_data.db')

# Geting all the data
# querying the database to get player information form gameweek table
query = '''
    SELECT p.player_id, p.name, p.position, p.team,
           g.gameweek, g.minutes, g.goals_scored, g.assists,
           g.clean_sheets, g.bonus, g.total_points
    FROM gameweek_data g
    JOIN players p ON g.player_id = p.player_id
    ORDER BY p.player_id, g.gameweek
'''

# storing the data that was got from the query in a dataframe(table)
df = pd.read_sql_query(query, conn)
# closing the connection
conn.close()

# loading the whole dataframe/table
# Count number of players in the dataframe & the gameweek
print(f"Loaded {len(df)} records")
print(f"Players: {df['player_id'].nunique()}")
print(f"Gameweeks: {df['gameweek'].min()} to {df['gameweek'].max()}")
# printing out some of the dataframe table
print(df.head())

Loaded 15760 records
Players: 795
Gameweeks: 1 to 21
   player_id  name position     team  gameweek  minutes  goals_scored  \
0          1  Raya       GK  Arsenal         1       90             0   
1          1  Raya       GK  Arsenal         2       90             0   
2          1  Raya       GK  Arsenal         3       90             0   
3          1  Raya       GK  Arsenal         4       90             0   
4          1  Raya       GK  Arsenal         5       90             0   

   assists  clean_sheets  bonus  total_points  
0        0             1      3            10  
1        0             1      0             6  
2        0             0      0             2  
3        0             1      0             6  
4        0             0      0             2  


In [24]:
# Feature for rolling Averages 
# gets the players recent form from the lat 5 games
def rollingAVG(df, window=5):
    """ 
    Calculating the rolling average of total points for each player

    DataFrame with player data
    window number od previous games to average

    returns the average rolling average points
    """
    # window set in the parameter of the function
    print(f"Rolling average - last {window} games")

    # Sorting the data by player & gameweek
    df = df.sort_values(['player_id', 'gameweek'])
    
    # Calculating the rolling averages for each player, total points column, last 5 games
    # rolling average is calculated after the last 5 games 
    # .mean calculates the average
    # .shift(1) uses the previous games
    df['rolling_avg_points'] = df.groupby('player_id')['total_points'].transform(
        lambda x: x.rolling(window=window, min_periods=window).mean().shift(1)
    )

    # filling nan values with 0
    # e.g. gw 1-5 has no rolling average as not enough games have been played 
    # nan would break ml models so it is replaced with a 0 
    df['rolling_avg_points'] = df['rolling_avg_points'].fillna(0)
    
    # returning the daataframe with rolling averages 
    return df

# storing the data in a dataframe
df = rollingAVG(df, window=5)

# showing how the rolling averages works 
print("\nRolling average data")
print("\nHaaland's form:")
eg = df[df['name'].str.contains('Haaland', na=False)][
    ['name', 'gameweek', 'total_points', 'rolling_avg_points']
].head(10)
print(eg)

# checking the correlation between rolling average & points scored
# looking to see that the players with good form socre more points
print("\nCorrelation with points")
corr = df['rolling_avg_points'].corr(df['total_points'])
print(f"r = {corr:.3f}")

if corr > 0.5:
    print("Strong correlation, good form predicts more points")
elif corr > 0.3:
    print("Moderate correlation, form is a useful predictor")
else:
    print("Weak correlation")

Rolling average - last 5 games

Rolling average data

Haaland's form:
         name  gameweek  total_points  rolling_avg_points
9009  Haaland         1            13                 0.0
9010  Haaland         2             2                 0.0
9011  Haaland         3             9                 0.0
9012  Haaland         4            13                 0.0
9013  Haaland         5             9                 0.0
9014  Haaland         6            16                 9.2
9015  Haaland         7             8                 9.8
9016  Haaland         8            13                11.0
9017  Haaland         9             2                11.8
9018  Haaland        10            13                 9.6

Correlation with points
r = 0.411
Moderate correlation, form is a useful predictor


In [25]:
# saving the engineering features
print("\nFeatures saved")

# saving to a csv file 
# index does not allow it to add extra row numbers 
df.to_csv('fpl_features.csv', index=False)
print("Saving features to 'fpl_features.csv'")

# saving to the database for fast querying in Ml training
# if it exists replace new with the old data
conn = sqlite3.connect('fpl_data.db')
df.to_sql('features', conn, if_exists='replace', index=False)
conn.close()
print("Saved to database features")

print(f"\nFeature engineering done")
print(f"Total records: {len(df)}")
print(f"Feature: rollingAverages - last 5 games")



Features saved
Saving features to 'fpl_features.csv'
Saved to database features

Feature engineering done
Total records: 15760
Feature: rollingAverages - last 5 games


In [26]:
# feature that predicts player performance based on
# the oppenet the face on the upcomig gameweek
# players will predict higher points if they face a weak team

# getting the team name from the api 
# mapping the team id to the team name
print("Getting team mappings")

# getting data that holds all the teams info
bootstrap_url = "https://fantasy.premierleague.com/api/bootstrap-static/"
bootstrap_data = requests.get(bootstrap_url).json()

# creating a dictionary to map the team id's to the team name 
GetTeamName = {team['id']: team['name'] for team in bootstrap_data['teams']}

# printing all the teams
print(f"{len(GetTeamName)} teams")

Getting team mappings
20 teams


In [27]:
# getting the fixtures of what team plays who
print("Getting fixtures")

# getting the fixtures from the FPL API
fixtures_url = "https://fantasy.premierleague.com/api/fixtures/"
fixtures_data = requests.get(fixtures_url).json()

# creating a fixtures dataframe
# to store the data that was got from the query in a dataframe(table)
fixtures_list = []
for fixture in fixtures_data:
    # only has the games that are assigned event
    if fixture['event']:
        # home team's view
        fixtures_list.append({
            'gameweek': fixture['event'],
            'team': GetTeamName[fixture['team_h']],      
            'opponent': GetTeamName[fixture['team_a']],  
            'is_home': True
        })
        # away team's view
        fixtures_list.append({
            'gameweek': fixture['event'],
            'team': GetTeamName[fixture['team_a']], 
            'opponent': GetTeamName[fixture['team_h']],
            'is_home': False
        })

# converting the list to a dataframe table
fixtures_df = pd.DataFrame(fixtures_list)

# amount of fixtures it gets 
# printing out the fixtures for gameweek 1
print(f"{len(fixtures_df)} fixtures")
print("\nfixtures GW1")
print(fixtures_df[fixtures_df['gameweek'] == 1].head(10))

Getting fixtures
760 fixtures

fixtures GW1
   gameweek         team     opponent  is_home
0         1    Liverpool  Bournemouth     True
1         1  Bournemouth    Liverpool    False
2         1  Aston Villa    Newcastle     True
3         1    Newcastle  Aston Villa    False
4         1     Brighton       Fulham     True
5         1       Fulham     Brighton    False
6         1        Spurs      Burnley     True
7         1      Burnley        Spurs    False
8         1   Sunderland     West Ham     True
9         1     West Ham   Sunderland    False


In [28]:
# Seeing how strong a teams defeense is
# a team with a weak defense has a high difficulty rating & vice versa

# get the player data with rolling averages
conn = sqlite3.connect('fpl_data.db')
df = pd.read_sql_query("SELECT * FROM features", conn)
conn.close()

# calculate the average points scored against each team
# the teams that conceded more points are easier to play 
teamDefense = df.groupby('team')['total_points'].mean().reset_index()
teamDefense.columns = ['team', 'avg_points_conceded']

# creating a difficulty rating 1 - 10 for each team
# 1 means the team has a strong defense & dosent concede many points
# while 10 means the team has a weak defense* concedes a lot of points
minPts = teamDefense['avg_points_conceded'].min()
maxPts = teamDefense['avg_points_conceded'].max()

# making the scale from 1-10
teamDefense['difficulty_rating'] = (
    ((maxPts - teamDefense['avg_points_conceded']) / (maxPts - minPts)) * 9 + 1
).round(1)


# for every game, put a difficulty rating on oppopnent based on their defense
# creating a dictionary with the team name and difficulty
opponentDifficulty = dict(zip(teamDefense['team'], teamDefense['difficulty_rating']))

# adding the opponent difficulty to the fixtures
fixtures_df['opponent_difficulty'] = fixtures_df['opponent'].map(opponentDifficulty)

# printing out the fixtures dataframe table for arsenal of their first 5 games
# with the opponent difficulty 
print(fixtures_df[fixtures_df['team'] == 'Arsenal'].head())



    gameweek     team       opponent  is_home  opponent_difficulty
17         1  Arsenal        Man Utd    False                  7.0
30         2  Arsenal          Leeds     True                  5.5
57         3  Arsenal      Liverpool    False                  4.7
60         4  Arsenal  Nott'm Forest     True                  7.9
98         5  Arsenal       Man City     True                  1.0


In [29]:
# adding the oponent difficulty rating to each players gameweek data
# combineing the fixtures with the player data
df = df.merge(
    fixtures_df[['gameweek', 'team', 'opponent', 'opponent_difficulty']], 
    # matching by gameweek and team
    on=['gameweek', 'team'],
    # keeing all the players even if no fixture is found
    how='left'
)

# make any missing values default to 5.0 difficulty
# handles edge cases where data is missing
df['opponent_difficulty'] = df['opponent_difficulty'].fillna(5.0)


# e.g. for haaland
print("\nHaaland's fixtures with opponent difficulty")
haaland = df[df['name'].str.contains('Haaland', na=False)][
    ['name', 'gameweek', 'team', 'opponent', 'total_points', 
     'rolling_avg_points', 'opponent_difficulty']
].head(10)
print(haaland)

# checking the correlation between oponent difficulty & points scored
# looking to see that the players that face easy opponents score more points
print("\nCorrelation with points:")
corr = df['opponent_difficulty'].corr(df['total_points'])
print(f"r = {corr:.3f}")

if corr > 0.0:
    print("+ correlation shows easier opponents lead to more points!")
elif corr < 0.0:
    print("- correlation shows an unexpected pattern")
else:
    print("Weak correlation means the feature is not implemented properly")

# saving the dataset with all features
print("\nSaving features...")

# saving to the database for fast querying in Ml training
# if it exists replace new with the old data
conn = sqlite3.connect('fpl_data.db')
df.to_sql('features', conn, if_exists='replace', index=False)
conn.close()

print("Saved features")
print(f"Total records: {len(df)}")


Haaland's fixtures with opponent difficulty
         name  gameweek      team     opponent  total_points  \
9009  Haaland         1  Man City       Wolves            13   
9010  Haaland         2  Man City        Spurs             2   
9011  Haaland         3  Man City     Brighton             9   
9012  Haaland         4  Man City      Man Utd            13   
9013  Haaland         5  Man City      Arsenal             9   
9014  Haaland         6  Man City      Burnley            16   
9015  Haaland         7  Man City    Brentford             8   
9016  Haaland         8  Man City      Everton            13   
9017  Haaland         9  Man City  Aston Villa             2   
9018  Haaland        10  Man City  Bournemouth            13   

      rolling_avg_points  opponent_difficulty  
9009                 0.0                 10.0  
9010                 0.0                  6.6  
9011                 0.0                  8.2  
9012                 0.0                  7.0  
9013      

Saved features
Total records: 15760


In [30]:
# feature to show that the minutes a player plays impacts their score
# a player who dosent play will score 0 points
# printing the minutes stats
print("Minutes stats")
print(df['minutes'].describe())

# checking the correlation between minutes played & points scored
# looking to see that players who play more score more points
print("\nCorrelation with points:")
corr = df['minutes'].corr(df['total_points'])
print(f"r = {corr:.3f}")

if corr > 0.5:
    print("Strong correlation, the more minutes played the more points scored")
else:
    print("Weak correlation")

# showing that the minutes played works
print("\nHaaland's minutes played v points scored")
haaland_minutes = df[df['name'].str.contains('Haaland', na=False)][
    ['name', 'gameweek', 'minutes', 'total_points']
].head(10)
print(haaland_minutes)


Minutes stats
count    15760.000000
mean        26.247462
std         37.676007
min          0.000000
25%          0.000000
50%          0.000000
75%         68.000000
max         90.000000
Name: minutes, dtype: float64

Correlation with points:
r = 0.682
Strong correlation, the more minutes played the more points scored

Haaland's minutes played v points scored
         name  gameweek  minutes  total_points
9009  Haaland         1       72            13
9010  Haaland         2       90             2
9011  Haaland         3       90             9
9012  Haaland         4       86            13
9013  Haaland         5       75             9
9014  Haaland         6       90            16
9015  Haaland         7       90             8
9016  Haaland         8       90            13
9017  Haaland         9       90             2
9018  Haaland        10       81            13
